##### Установка сторонних модулей через pip

In [68]:
pip install nltk numpy pymorphy2

Note: you may need to restart the kernel to use updated packages.


##### Импорт библиотек

In [69]:
import json
import re
import nltk
import pymorphy2
import pickle

nltk.download('punkt');

PATH = './data/data.json'
PATH_OBJECT = './data/data'

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##### Токенизация/первичный анализ
1. Токенизация текста
2. Приведение в нижний регистр
2. Исключение спец символов и ангийского текста
4. Исключение частей речи не несущих смысловой нагрузки (https://pymorphy2.readthedocs.io/en/latest/user/grammemes.html#russian-pos)

In [70]:
def tokenize(text):
    morph = pymorphy2.MorphAnalyzer()
    sentences = nltk.word_tokenize(text, language="russian")
    tokens = []
    for val in sentences:
        val = val.lower()
        if re.search(r'\d', val) or re.search(r'[а-я]+', val):
            p = morph.parse(val)[0]
            if p.tag.POS != 'INTJ' and p.tag.POS != 'PRCL' and p.tag.POS != 'CONJ' and p.tag.POS != 'PREP' and p.tag.POS != 'PRED' and p.tag.POS != 'NPRO':
                tokens.append(val)
    return tokens

##### Удаление HTML тегов

In [71]:
def removeHtml(html):
    teg_re = re.compile(r'<[^>]+>')
    return teg_re.sub(' ', html)

##### Чтение объекта из файла

In [72]:
def readObjectFromFile():
    filehandler = open(PATH_OBJECT, 'rb') 
    return pickle.load(filehandler)

##### Запись объекта в файл

In [78]:
def writeObjectFromFile(data):
    filehandler = open(PATH_OBJECT, 'wb') 
    pickle.dump(data, filehandler)

##### Создание массива текстов textsArr

In [75]:
%%time
textsArr = []
file = open(PATH, 'r')
line = file.readline()
while line:
    loaded_json = json.loads(line)
    text = loaded_json['Description']
    textsArr.append(text)
    line = file.readline()
file.close()

print("Count texts: " + str(len(textsArr)))

Count texts: 5336
CPU times: user 97.1 ms, sys: 5.14 ms, total: 102 ms
Wall time: 98.5 ms


##### Создание data - массив массива токенов

In [76]:
%%time
data = []
for val in textsArr:
    val = removeHtml(val)
    val = tokenize(val)
    data.append(val)

CPU times: user 5min 30s, sys: 13.2 s, total: 5min 43s
Wall time: 5min 43s


##### Запись предобработанного объекта в файл

In [79]:
writeObjectFromFile(data)